# 📘 SECTION 0 — Header & Licensing (ADK Style)

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


<!-- ##### Copyright 2025 Google LLC. -->

## 🧠 Smart Research Assistant — Multi-Agent ADK Project  
### *A multi-agent system with parallel + sequential agents, custom tools, built-in tools, sessions, memory, and context engineering.*

This notebook implements a **complete multi-agent system** using the  
**Google Agent Development Kit (ADK)**, following the architectural and coding
patterns from the official Day-1 → Day-5 ADK notebooks.

You will learn how to:

- Build **LLM-powered agents**  
- Run agents in **sequential and parallel** workflows  
- Use **built-in ADK tools** (Google Search, Python Code Execution)  
- Build **custom FunctionTools**  
- Manage context with **sessions, memory, and compaction**  
- Combine agent outputs to generate a structured **research brief** including:  
  - Topic understanding  
  - Web findings  
  - Conceptual explanation  
  - 5 strong points  
  - 5 weak points / gaps  
  - Synthesized final summary  

Let's get started. 🚀


# 📘 SECTION 1 — Project Overview
Smart Research Assistant — Multi-Agent ADK System

## 🚀 Section 1: Project Overview

Welcome to the **Smart Research Assistant**, a fully ADK-compliant multi-agent system
designed to generate high-quality research briefs using:

- **Sequential and parallel agents**  
- **LLM-powered reasoning**  
- **Built-in ADK tools** (Google Search, Code Execution)  
- **Custom FunctionTools** for summarization and structure  
- **Sessions, memory, and context compaction**  
- **Structured, JSON-friendly outputs**  

---

### 🎯 What this agent system does

Given a topic such as:

> *"Sleep disorder detection using multimodal signals"*

The system will:

1. **Understand** the user's query  
2. **Extract keywords** and contextual information  
3. Run **parallel agents** to:
   - Search the web for recent findings  
   - Produce conceptual and theoretical background  
4. Identify:
   - **5 strong points** in existing research  
   - **5 weak points or gaps**  
5. **Synthesize everything** into a structured, well-organized research brief  

---

### 🧩 Key Features You Will Implement

- **Multi-agent architecture**  
  - Query Understanding Agent  
  - Web Research Agent (parallel)  
  - Knowledge Explainer Agent (parallel)  
  - Strength–Weakness Analyzer  
  - Final Synthesizer Agent  

- **Parallel execution** with `asyncio.gather`  
- **Sessions & Memory**  
  - Persistent conversation context  
  - State tracking  
  - Memory over multiple calls  

- **Context Engineering**
  - Context compaction  
  - Summaries for long contexts  
  - Relevance-based memory filtering  

- **Tools**
  - Google Search Tool  
  - Python Code Execution Tool  
  - Custom FunctionTools (keyword extractor, bullet formatter, etc.)  

---

### 🧱 How this notebook is structured

This notebook follows the **same instructional design style as the ADK daily notebooks**:

1. Setup  
2. Architecture diagram  
3. Sessions & memory  
4. Tools  
5. Agents  
6. Root Orchestrator  
7. Runner  
8. Demo  
9. Appendix  

Each section is implemented with:
- Configuration cells  
- Code blocks  
- Explanatory Markdown  
- ADK-style comments and structure  

---

### 🎉 By the end of this notebook

You will have a *fully functional*, *deployable*, *extensible*  
**multi-agent research assistant** built with the Google ADK.

Let's move to the setup.


# 📘 SECTION 2 — Setup
Install dependencies, import libraries, configure API keys, and set core settings.

## ⚙️ Section 2: Setup

In this section, we will:

1. Install the necessary dependencies  
2. Import ADK and Python libraries  
3. Configure your Gemini API Key  
4. Set up model and retry configurations  
5. Define notebook-level constants  

These steps follow the same environment initialization pattern used
throughout the official ADK Day 1–5 notebooks.


### 2.1 Install Dependencies

In [2]:
# # @title Install dependencies
# # This cell installs the Google Agent Development Kit (ADK) and other required libraries.

# !pip install -q google-generativeai
# !pip install -q google-adk
# !pip install -q python-dotenv


### 2.2 Import Libraries

In [3]:
# # @title Import core libraries

# import os
# import json
# import asyncio
# from typing import Any, Dict, List

# # ADK components
# from google.adk.agents import Agent
# from google.adk.runners import InMemoryRunner
# from google.adk.sessions import InMemorySessionService
# from google.adk.tools import FunctionTool, PythonCodeExecutor, GoogleSearchTool
# from google.adk.types import HttpRetryOptions
# from google.adk.models import google_llm



# Core Python Libraries
import os
import sys
import json
import random
import logging
import datetime
from pathlib import Path
from typing import List, Dict, Union, Any, Optional
from pprint import pprint

# Data Handling
import pandas as pd
import numpy as np

# Kaggle Resources
from kaggle_secrets import UserSecretsClient

# Call synchronisation 
import asyncio

# Google ADK Modules (Standard Day 1 - Day 5 Patterns)
try:
    import google.adk 
    from google.adk.agents import LlmAgent, Agent, SequentialAgent, ParallelAgent, LoopAgent        
    from google.adk.models.google_llm import Gemini
    from google.adk.runners import Runner, InMemoryRunner
    from google.adk.sessions import InMemorySessionService, DatabaseSessionService #, InMemoryMemoryService 
    from google.adk.memory import InMemoryMemoryService
    from google.adk.tools import load_memory, preload_memory
    from google.adk.tools import google_search, FunctionTool, AgentTool, ToolContext, BaseTool 
    from google.adk.tools.tool_context import ToolContext
    from google.adk.code_executors import BuiltInCodeExecutor
    from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
    from google.genai import types
    from google.adk.apps.app import App, ResumabilityConfig, EventsCompactionConfig
    from google.adk.tools.function_tool import FunctionTool
    
    from google.genai import types
    import google.generativeai as genai

    # A2A / Production Modules (Day 5 Pattern)
    from google.adk.agents.remote_a2a_agent import RemoteA2aAgent, AGENT_CARD_WELL_KNOWN_PATH
    from google.adk.a2a.utils.agent_to_a2a import to_a2a

    print(f"google.adk.__version__: {google.adk.__version__}")
    print(f"google.adk.__file__: {google.adk.__file__}")
    print("✅ Google ADK libraries imported successfully.")

except ImportError as e:
    print("❌ Error importing Google ADK. Please ensure the environment is configured correctly.")
    print(f"Details: {e}")

google.adk.__version__: 1.18.0
google.adk.__file__: /usr/local/lib/python3.11/dist-packages/google/adk/__init__.py
✅ Google ADK libraries imported successfully.


### 2.3 Configure Gemini API Key

In [4]:
try:
    # Attempt to retrieve the secret labeled 'GOOGLE_API_KEY'
    GOOGLE_API_KEY = UserSecretsClient().get_secret('GOOGLE_API_KEY')
    
    # Set it as an environment variable for ADK/Gemini to find
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    
    print("✅ API Key successfully loaded from Kaggle Secrets.")
    
except Exception as e:
    print("⚠️ Authentication Warning:")
    print("   Could not find 'GOOGLE_API_KEY' in Kaggle Secrets.")
    print("   Please go to 'Add-ons' -> 'Secrets' and add your key.")
    print(f"   Details: {e}")

✅ API Key successfully loaded from Kaggle Secrets.


### 2.4 Retry Configuration

In [5]:
# @title Retry configuration 
retry_config = types.HttpRetryOptions(
    attempts = 5, 
    exp_base = 2, 
    initial_delay = 1, 
    http_status_codes = [429, 500, 503, 504]
    )

retry_config


HttpRetryOptions(
  attempts=5,
  exp_base=2.0,
  http_status_codes=[
    429,
    500,
    503,
    504,
  ],
  initial_delay=1.0
)

### 2.5 Model Configuration

In [6]:
# @title Model configuration

# AGENT_MODEL = "gemini-2.5-flash-lite" #"gemini-2.5-flash",     # Fast, cost-efficient research model #"gemini-2.5-flash" "gemini-2.5-flash-lite"
# MODEL = Gemini(model=AGENT_MODEL)

# if retry_config: 
#     MODEL = Gemini(model=AGENT_MODEL, retry_options=retry_config)
# else:
#     MODEL = Gemini(model=AGENT_MODEL)

# MODEL

### 2.6 Notebook Constants

In [7]:
# @title Notebook constants

MAX_CONTEXT_TURNS = 8            # Maximum turns to keep before compaction
MAX_HISTORY_KEEP = 3             # Number of recent items preserved during compaction
SUMMARY_MODEL = "gemini-2.5-flash"

print("Setup complete. ADK environment initialized.")


Setup complete. ADK environment initialized.


# 📘 SECTION 3 — Project Architecture Overview
Understanding the multi-agent workflow before implementation

## 🧱 Section 3: Project Architecture Overview

Before implementing the agents, tools, memory, and orchestrator,
it's important to understand **how the Smart Research Assistant works end-to-end**.

This project uses a **hybrid multi-agent architecture** that combines:

- **Sequential execution** (step-by-step reasoning and synthesis)
- **Parallel execution** (simultaneous research streams)
- **LLM-driven agents**
- **Custom + built-in tools**
- **Sessions + memory**
- **Context compaction**

Below is an overview of the data and control flow.


### 3.1 High-Level Workflow Diagram

In [8]:
# 🧩 Multi-Agent Workflow (Sequential + Parallel)

"""
User Query
    │
    ▼
📘 Query Understanding Agent  (LLM + Memory)
    - Interprets the topic
    - Extracts keywords
    - Reads previous session memory
    - Produces structured intent JSON
    │
    ▼
===================== PARALLEL EXECUTION =====================
│                                                          │
▼                                                          ▼
🔍 Web Research Agent                                      🧠 Knowledge Explainer Agent
- Uses Google Search Tool                                 - LLM conceptual explanation
- Finds recent findings                                   - Expands topics and theory
- Extracts insights                                       - Summarizes academic context
│                                                          │
===================== RESULTS MERGE BACK ======================
    │
    ▼
📊 Strength–Weakness Agent
    - Produces 5 strong points
    - Produces 5 weak points (gaps)
    - Applies context compaction on parallel outputs
    │
    ▼
📝 Research Synthesizer Agent
    - Merges all results
    - Ensures structured final JSON output
    - Updates session memory
    │
    ▼
🎉 Final Research Brief (JSON + formatted text)
"""

""

''

### 3.2 Key Architectural Concepts

### 🔸 Agents (LLM-powered)
Each agent is a subclass of `Agent` and implements an async `run()` method.
Agents may:
- Call tools
- Perform reasoning
- Transform or merge results
- Update memory

### 🔸 Tools
We use:
- Google Search Tool (built-in)
- Python Code Execution (optional)
- Custom FunctionTools for:
  - keyword extraction
  - summarization
  - bullet list formatting
  - gap identification

### 🔸 Sessions & Memory
Session data persists across calls using:

`InMemorySessionService()`

To avoid runaway context, we implement:

**context compaction**, where:
- only the last few turns are kept fully
- older turns are summarized
- irrelevant historical content is removed

### 🔸 Parallelism
Two agents are run concurrently:

- Web Research Agent  
- Knowledge Explainer Agent  

using `asyncio.gather()`.

### 🔸 Final Synthesis
A final agent merges:
- web findings
- theoretical background
- strengths & weaknesses
into a single structured JSON object.


### 3.3 Architecture Summary

The Smart Research Assistant follows a **progressive refinement workflow**:

1. Understand the query  
2. Expand the topic using two parallel research paths  
3. Evaluate strong/weak points  
4. Compress context  
5. Produce final structured output  

This mirrors real academic research workflows and demonstrates
all ADK concepts required for the Capstone project.


# 📘 SECTION 4 — Sessions & Memory
State management, session continuity, and context compaction.

## 🧠 Section 4: Sessions & Memory

Large Language Model (LLM)–powered agents work best when they have access to
**persistent state across multiple turns**. In the Google ADK, this is handled
through a **Session Service** that stores agent inputs, outputs, and custom metadata.

In this project, sessions enable the Smart Research Assistant to:

- Remember previous queries  
- Maintain extracted keywords or constraints  
- Preserve research context  
- Apply context compaction to avoid runaway prompt growth  

We will implement:

1. `InMemorySessionService` — ADK's built-in lightweight session layer  
2. A custom `MemoryManager` — handles saving, retrieving, and compacting memory  
3. Context compaction rules — mirrors Day-3B’s pattern  


### 4.1 Session Service Initialization

In [9]:
# @title Initialize Session Service

# The session service stores conversation-level state that persists across calls.
session_service = InMemorySessionService()

print("Session service initialized.")


Session service initialized.


### 4.2 MemoryManager Class

This class mirrors the design philosophy of the ADK Day-3B “Agent Memory” notebook, including:

- Turn-based memory items

- Summarization of older history

- Keeping only the most relevant parts

- Optional compaction triggers

In [10]:
# @title Memory Manager Implementation

class MemoryManager:
    """
    Handles storage and compaction of memory for each session.

    The memory structure mirrors best practices from the ADK Day-3 notebooks:
    - Keep recent turns fully
    - Summarize older turns
    - Remove irrelevant or low-value content
    """

    def __init__(self, max_turns=MAX_CONTEXT_TURNS, keep_last=MAX_HISTORY_KEEP):
        self.max_turns = max_turns
        self.keep_last = keep_last

    def load(self, session_state: dict) -> List[dict]:
        """Retrieve memory list from session state."""
        return session_state.get("memory", [])

    def save(self, session_state: dict, memory: List[dict]):
        """Persist updated memory back to the session."""
        session_state["memory"] = memory

    async def compact(self, memory: List[dict], llm_model) -> List[dict]:
        """
        Apply context compaction:
        - If memory is short, return as-is
        - If it exceeds max_turns, summarize older items using an LLM
        """
        if len(memory) <= self.max_turns:
            return memory

        # Step 1: Keep the last N items unchanged
        recent_items = memory[-self.keep_last:]

        # Step 2: Summarize older items
        old_items = memory[:-self.keep_last]
        old_text = "\n".join([item["content"] for item in old_items])

        summary_prompt = (
            "Summarize the following conversation history into a concise "
            "set of key points that preserve meaning but remove redundancy:\n\n"
            f"{old_text}"
        )

        summary = await llm_model(summary_prompt)

        # Step 3: Replace old items with a single summary item
        compacted = [{"type": "summary", "content": summary}] + recent_items

        return compacted


### 4.3 Helper Functions for Memory Operations

These helpers mimic patterns in ADK notebooks:
clean, readable, and explicitly tied to the session state.

In [11]:
# @title Memory helper functions

async def load_memory(session_id: str) -> List[dict]:
    """Retrieve memory for a given session."""
    session_state = session_service.get(session_id)
    memory = memory_manager.load(session_state)
    return memory


async def save_memory(session_id: str, memory: List[dict]):
    """Save updated memory after compaction or new entries."""
    session_state = session_service.get(session_id)
    memory_manager.save(session_state, memory)
    session_service.set(session_id, session_state)


async def add_memory_item(session_id: str, item: dict, llm_model):
    """Add a new item and apply compaction if necessary."""
    memory = await load_memory(session_id)
    memory.append(item)
    memory = await memory_manager.compact(memory, llm_model)
    await save_memory(session_id, memory)


### 4.4 Instantiate MemoryManager

In [12]:
# @title Instantiate Memory Manager

memory_manager = MemoryManager(
    max_turns=MAX_CONTEXT_TURNS,
    keep_last=MAX_HISTORY_KEEP
)

print("Memory manager initialized with compaction rules:")
print(f"- Max turns: {MAX_CONTEXT_TURNS}")
print(f"- Keep last: {MAX_HISTORY_KEEP}")


Memory manager initialized with compaction rules:
- Max turns: 8
- Keep last: 3



### 4.5: Helper functions

This helper function manages a complete conversation session, handling session creation/retrieval, query processing, and response streaming.


In [13]:
# @helper function for running session 

async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


# 📘 SECTION 5 — Tools Setup
Built-in tools + custom FunctionTools for research analysis

## 🔧 Section 5: Tools Setup

Agents in ADK become far more capable when they can call tools.

In this project, tools enable:
- Web research (Google Search)
- Text summarization
- Keyword extraction
- Strength/weakness detection
- Bullet formatting
- JSON cleaning

Following ADK best practices (Day-2 notebooks):
- Tools should be **stateless**
- Tools should be **atomic** (do one thing well)
- Tools should return **dict-like outputs**
- Tools should declare name + description for LLM routing


### 5.1 Google Search Tool (Built-in)

google_search built-in tool to be Used by the Web Research Agent.

In [14]:
# # @title Google Search Tool

# def google_search_agent_tool(gsa_name, gsa_instruction): 
#     google_search_tool = Agent(
#         name=gsa_name,
#         model=Gemini(model=SUMMARY_MODEL, retry_options=retry_config),
#         instruction=gsa_instruction,
#         tools=[google_search],     # ← This is the tool
#     )
#     return google_search_tool

# print("Google Search Tool initialized.")


# # @agent Agent for Google search

# researcher_economic = google_search_agent_tool(gsa_name="ResearcherEconomic", gsa_instruction=""" You are an expert reseracher that searches on internet (google_search tool) for specific research topic. 
# You read first 10 papers to find out the summary, research findings, strong points from the findings and research gaps. 
# After google search read each paper (if freely available on internet or else read the abstract only) in details to find out the above information and summarise them. 
# You must read the paper as a positive reader focusing mainly from financial and application perspective. 
# """) 

# researcher_technical = google_search_agent_tool(gsa_name="ResearcherTechnical", gsa_instruction=""" You are an expert reseracher that searches on internet (google_search tool) for specific research topic. 
# You read first 15 papers to find out the summary, research findings, strong points from the findings and research gaps. 
# After google search read each paper (if freely available on internet or else read the abstract only) in details to find out the above information and summarise them. 
# You must read the paper as a positive reader focusing mainly from technical and information technology perspective. 
# """) 

# researcher_economic, researcher_technical 


# @title Google Search Tool 
SEARCH_TOOL = google_search 

SEARCH_TOOL


### 5.2 Keyword Extraction Tool (FunctionTool)

Matches ADK practice: small, stateless, atomic.

In [15]:
# @title Keyword Extraction Tool

def extract_keywords(text: str) -> dict:
    """
    A simple heuristic keyword extractor.
    ADK best practices recommend atomic, deterministic tools.
    """
    words = [w.strip(".,()") for w in text.lower().split()]
    stopwords = {"the","and","or","to","a","is","in","of","for","on","with","using"}
    keywords = [w for w in words if w not in stopwords and len(w) > 4]
    keywords = list(set(keywords))  # ensure uniqueness
    return {"keywords": keywords}

print("Keyword Extraction Tool initialized.")


keyword_tool = FunctionTool(
    # name="extract_keywords",
    # description="Extracts keywords from input text using simple heuristics.",
    func=extract_keywords,
)

keyword_tool


Keyword Extraction Tool initialized.


### 5.3 Bullet List Formatting Tool (FunctionTool)

Used later by the Strength–Weakness Agent.

In [16]:
# @title Bullet List Formatting Tool

def bullet_format(items: list) -> dict:
    """
    Format a list of items as clean bullet points.
    """
    formatted = "\n".join([f"- {item}" for item in items])
    return {"bullets": formatted}

print("Bullet Formatting Tool initialized.")

bullet_tool = FunctionTool(
    # name="bullet_format",
    # description="Formats a list of strings into Markdown-style bullet points.",
    func=bullet_format,
)

bullet_tool 


Bullet Formatting Tool initialized.


### 5.4 Strength/Weakness Heuristic Extraction Tool (FunctionTool)

This tool is used before LLM refinement — exactly like Day-2B’s pattern of
“weak heuristic → LLM agent for refinement”.

In [17]:
# @title Strength/Weakness Extraction Tool

def extract_strengths_weaknesses(text: str) -> dict:
    """
    Heuristic extraction of candidate strengths and weaknesses.
    The LLM agent will refine these later.
    """
    sentences = [s.strip() for s in text.split(".") if len(s.strip()) > 8]

    strengths = sentences[:5] if len(sentences) >= 5 else sentences
    weaknesses = sentences[-5:] if len(sentences) >= 5 else sentences

    return {
        "strengths": strengths,
        "weaknesses": weaknesses,
    }

print("Strength/Weakness Tool initialized.")

strength_gap_tool = FunctionTool(
    # name="extract_strengths_weaknesses",
    # description="Extracts candidate strengths and weaknesses from raw text.",
    func=extract_strengths_weaknesses,
)

strength_gap_tool 


Strength/Weakness Tool initialized.


### 5.5 Tool Registry (ADK Style)

This is exactly like the pattern used in Day-2 notebooks:
Centralize tools so that root agents can inject them.

In [18]:
# @title Tool Registry

TOOLS = {
    "web_search": SEARCH_TOOL,               # ADK-provided google_search tool
    "extract_keywords": keyword_tool,
    "bullet_format": bullet_tool,
    "strength_gap_tool": strength_gap_tool,
}

TOOLS


{'web_search': <google.adk.tools.google_search_tool.GoogleSearchTool at 0x7ccf698ecbd0>,
 'extract_keywords': <google.adk.tools.function_tool.FunctionTool at 0x7ccf67962910>,
 'bullet_format': <google.adk.tools.function_tool.FunctionTool at 0x7ccf679895d0>,
 'strength_gap_tool': <google.adk.tools.function_tool.FunctionTool at 0x7ccf85b82e90>}

# 📘 SECTION 6 — Agent Implementations
LLM-powered agents for a sequential + parallel workflow

## 🤖 Section 6: Agent Implementations

We now define all LLM-powered agents used in the Smart Research Assistant.

Each agent follows the ADK agent pattern:

- Uses a `Gemini` model
- Has a clear instruction block
- Returns structured JSON-like dicts
- May call tools (e.g., web_search)
- May use memory/context to refine behavior
- Implements `async run()` as recommended in ADK tutorials

We implement 5 agents:

1. Query Understanding Agent  
2. Web Research Agent (parallel)  
3. Knowledge Explainer Agent (parallel)  
4. Strength–Weakness Agent  
5. Research Synthesizer Agent  


### 6.1 Query Understanding Agent

Extracts topic, keywords, and relevant memory.

This one does not use tools.
It uses LLM reasoning + memory input.

In [19]:
# @title QueryUnderstandingAgent

class QueryUnderstandingAgent(Agent):
    def __init__(self, model):
        super().__init__(
            name="QueryUnderstandingAgent",
            model=model,
            instruction="""
You are a Query Understanding Agent.

Your goals:
1. Interpret the user's research question clearly.
2. Extract the core topic.
3. Identify relevant keywords.
4. Incorporate prior session memory summaries (if available).
5. Produce structured JSON output with:
   - topic
   - keywords
   - context_summary
""",
            output_key="query_understanding",
        )

    async def run(self, user_query: str, memory_summary: str = ""):
        prompt = f"""
USER QUERY:
{user_query}

MEMORY SUMMARY:
{memory_summary}

TASK:
Return a JSON object with:
- "topic": short topic description
- "keywords": list of important terms
- "context_summary": 1–2 sentence overall interpretation
"""
        response = await self.model(prompt)
        return {"topic": user_query, "analysis": response}


### 6.2 Web Research Agent (Parallel Agent #1)

Uses google_search to retrieve external info.

This matches the pattern used in the Day-1B example where they define:

tools=[google_search]

🌟 Important: we wrap the results using ADK's AgentTool pattern only when chaining agents, but here we call the tool directly inside run().

In [20]:
# @title WebResearchAgent

class WebResearchAgent(Agent):
    def __init__(self, model, search_tool):
        super().__init__(
            name="WebResearchAgent",
            model=model,
            instruction="""
You are a Web Research Agent.

Use the `search tool` tool to gather:
- recent findings
- related topics
- short descriptions
- high-level insights

Return structured bullet points summarizing what you find.
""",
            tools=[search_tool],
            output_key="web_research",
        )

    async def run(self, topic: str):
        # The ADK google_search tool expects a dict input
        search_results = await self.tools["web_search"].call({"query": topic})

        prompt = f"""
Summarize the following search results into 5–7 concise points:

{search_results}
"""
        summary = await self.model(prompt)
        return {"web_findings": summary}


### 6.3 Knowledge Explainer Agent (Parallel Agent #2)

Provides conceptual, theoretical background.

In [21]:
# @title KnowledgeExplainerAgent

class KnowledgeExplainerAgent(Agent):
    def __init__(self, model):
        super().__init__(
            name="KnowledgeExplainerAgent",
            model=model,
            instruction="""
You are a Knowledge Explainer Agent.

Your job:
- Expand and clarify the topic.
- Provide conceptual, theoretical, or domain-specific insights.
- Assume the reader is familiar with research-level language.
- Return 2–4 short paragraphs of explanation.
""",
            output_key="concept_explanation",
        )

    async def run(self, topic: str):
        prompt = f"""
Explain the topic below in 2–4 short paragraphs:

TOPIC: {topic}

Keep it factual and academically neutral.
"""
        explanation = await self.model(prompt)
        return {"conceptual_background": explanation}


### 6.4 Strength–Weakness Agent

Uses heuristic FunctionTool + LLM refinement.

In [22]:
# @title StrengthWeaknessAgent

class StrengthWeaknessAgent(Agent):
    def __init__(self, model, strength_gap_tool, bullet_tool):
        super().__init__(
            name="StrengthWeaknessAgent",
            model=model,
            instruction="""
You are the Strength–Weakness Evaluation Agent.

Your job:
1. Take web findings + conceptual knowledge explanation.
2. Identify 5 strong points.
3. Identify 5 weak points (gaps).
4. Format them clearly.
""",
            tools=[strength_gap_tool, bullet_tool],
            output_key="strength_weakness",
        )

    async def run(self, combined_text: str):
        heuristic = await self.tools["strength_gap_tool"].call(
            {"text": combined_text}
        )

        prompt = f"""
Refine the following heuristic strengths/weaknesses.

HEURISTIC:
{heuristic}

Return exactly:
- "strengths": 5 bullet points
- "weaknesses": 5 bullet points
"""
        refined = await(self.model(prompt))

        return {"strengths_weaknesses": refined}


### 6.5 Research Synthesizer Agent

Final merging agent — similar to the aggregator agent in Day-1B.

In [23]:
# @title ResearchSynthesizerAgent

class ResearchSynthesizerAgent(Agent):
    def __init__(self, model):
        super().__init__(
            name="ResearchSynthesizerAgent",
            model=model,
            instruction="""
You are a Research Synthesizer Agent.

Combine:
- Query understanding
- Web research findings
- Conceptual explanation
- Strength–weakness analysis

Output a clean JSON with:
- topic
- conceptual_background
- strong_points
- weak_points
- overall_summary (2–3 paragraphs)
""",
            output_key="final_report",
        )

    async def run(self, pieces: dict):
        prompt = f"""
SYNTHESIZE THIS RESEARCH INFORMATION:

{pieces}

Produce a structured JSON object with:
1. "topic"
2. "conceptual_background"
3. "strong_points"
4. "weak_points"
5. "overall_summary"
"""
        result = await self.model(prompt)
        return {"final_report": result}


# 📘 SECTION 7 — Root Orchestrator Agent
A multi-step controller that governs the entire research pipeline

## 🧠 Section 7: Root Orchestrator Agent

This agent coordinates all other agents to produce the final research brief.

Workflow:
1. Load session memory (Section 4)
2. Run QueryUnderstandingAgent
3. Run WebResearchAgent and KnowledgeExplainerAgent in parallel
4. Combine outputs → StrengthWeaknessAgent
5. Merge all results → ResearchSynthesizerAgent
6. Save memory
7. Return final structured report

This follows the multi-step orchestration patterns demonstrated in:
- Day-1B (SequentialAgent, ParallelAgent)
- Day-5A (Custom Orchestrators + Agent-to-Agent Messaging)
- Day-5B (Realistic multi-agent flows)


### 7.1 Root Orchestrator Class

In [24]:
# @title Root Orchestrator Agent (FINAL FIX)

from typing import Any

class RootResearchOrchestrator(Agent):
    # 1. DECLARE FIELDS HERE so Pydantic allows them
    query_agent: Any = None
    web_agent: Any = None
    explain_agent: Any = None
    strength_agent: Any = None
    synth_agent: Any = None
    memory_manager: Any = None

    def __init__(
        self,
        model,
        query_agent,
        web_agent,
        explain_agent,
        strength_agent,
        synth_agent,
        memory_manager,
    ):
        super().__init__(
            name="RootResearchOrchestrator",
            model=model,
            instruction="""
You are the Root Research Orchestrator.
Your goal is to coordinate the research agents to answer the user query.
""",
            output_key="orchestrator_output",
        )

        # 2. Now you can assign them safely
        self.query_agent = query_agent
        self.web_agent = web_agent
        self.explain_agent = explain_agent
        self.strength_agent = strength_agent
        self.synth_agent = synth_agent
        self.memory_manager = memory_manager

    async def run(self, user_query: str, session_id: str = None):
        
        # 1. Load Memory
        session_state = session_service.get(session_id)
        # Use the injected memory_manager
        memory_items = self.memory_manager.load(session_state)
        memory_summary = "\n".join([m["content"] for m in memory_items]) if memory_items else ""

        # 2. Query Understanding
        q_result = await self.query_agent.run(
            user_query=user_query,
            memory_summary=memory_summary,
        )
        topic = q_result.get("topic", user_query)

        # 3. Parallel Agents (Web + Explanation)
        # We must await the tasks or gather them
        web_task = asyncio.create_task(self.web_agent.run(topic))
        explain_task = asyncio.create_task(self.explain_agent.run(topic))
        
        web_output, explain_output = await asyncio.gather(web_task, explain_task)

        combined_text = (
            f"WEB FINDINGS:\n{web_output}\n\n"
            f"EXPLANATION:\n{explain_output}"
        )

        # 4. Strength & Weakness
        sw_result = await self.strength_agent.run(combined_text)

        # 5. Synthesis
        pieces = {
            "query_understanding": q_result,
            "web_research": web_output,
            "conceptual_background": explain_output,
            "strengths_and_weaknesses": sw_result,
        }

        final_report = await self.synth_agent.run(pieces)

        # 6. Save memory
        # Note: In a notebook, 'add_memory_item' is global, which is fine.
        await add_memory_item(
            session_id,
            {"type": "research_turn", "content": user_query},
            llm_model=self.model,
        )

        return final_report

# 📘 SECTION 8 — Runner Setup
Connecting the orchestrator to the ADK runtime

In [25]:
# @title Instantiate All Agents

# Shared model configuration
shared_model = Gemini(
    model="gemini-2.5-flash",
    temperature=0.2,
    max_output_tokens=2048,
    retry_options=retry_config,
)

# Instantiate agents
query_agent = QueryUnderstandingAgent(model=shared_model)

web_agent = WebResearchAgent(
    model=shared_model,
    search_tool=SEARCH_TOOL,
)

explain_agent = KnowledgeExplainerAgent(
    model=shared_model
)

strength_agent = StrengthWeaknessAgent(
    model=shared_model,
    strength_gap_tool=strength_gap_tool,
    bullet_tool=bullet_tool
)

# strength_agent = StrengthWeaknessAgent(
#     model=shared_model,
#     web_findings_agent=web_agent,
#     knowledge_explainer_agent=explain_agent
# ) 

synth_agent = ResearchSynthesizerAgent(
    model=shared_model
)



## 🏃 Section 8: Runner Setup

We now create the ADK runner that executes the Root Orchestrator agent.

The ADK Runner:
- Manages input/output flow
- Tracks session state
- Provides a unified interface for calling the agent system

We use `InMemoryRunner` for notebook testing, exactly as shown in Day-1B and Day-2A.


### 8.1 Instantiate All Agents

Here, we wire together all agents built in Section 6.

### 8.2 Create the Root Orchestrator Agent

In [26]:
# @title Create Root Orchestrator Agent (CORRECTED)

# Instantiate the orchestrator with all dependencies
root_orchestrator = RootResearchOrchestrator(
    model=shared_model,
    query_agent=query_agent,
    web_agent=web_agent,
    explain_agent=explain_agent,
    strength_agent=strength_agent,
    synth_agent=synth_agent,
    memory_manager=memory_manager,
)

print("Root Orchestrator initialized correctly with sub-agents.")

Root Orchestrator initialized correctly with sub-agents.


### 8.3 Create the ADK Runner

This matches the ADK usage pattern:

runner = InMemoryRunner(agent=root_agent)


as seen in multiple daily notebooks (every Day-1B / Day-2 demo cell).

In [27]:
# @title Create InMemoryRunner

# runner = InMemoryRunner(root_orchestrator)
# print("InMemoryRunner is ready.")


# Define constants used throughout the notebook
APP_NAME = "SRA_App"
USER_ID = "Emran"

memory_service = (
    InMemoryMemoryService()
)

# Create Session Service
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=root_orchestrator,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

print("✅ Agent and Runner created with memory support!")


✅ Agent and Runner created with memory support!


# 📘 SECTION 9 — End-to-End Demo
Run the entire multi-agent system with a real research query

## 🧪 Section 9: End-to-End Demo

This section runs a full research request through the orchestrator using:
- Query Understanding Agent
- Parallel Web + Knowledge Agents
- Strength–Weakness Agent
- Research Synthesizer Agent
- Memory storage and compaction

The ADK `InMemoryRunner` will call our Root Orchestrator Agent.

You may modify the demo query below to test different topics.


### 9.1 Define a Demo Query

In [28]:
# @title Define a Demo Query

demo_query = "Sleep disorder detection using multimodal physiological signals"

session_id = "demo-session-001"   # Persistent session ID
print("Demo query ready:", demo_query)


Demo query ready: Sleep disorder detection using multimodal physiological signals


### 9.2 Run the Orchestrator via Runner

This matches the style used in:

result = runner.run("some input")


as seen in the daily notebooks.

In [29]:
# @title Run Full Research Pipeline


input_data = {
    "query": demo_query,
    "session_id": session_id,
    "query_agent": query_agent,
    "web_agent": web_agent,
    "explain_agent": explain_agent,
    "strength_agent": strength_agent,
    "synth_agent": synth_agent,
    "session_id": session_id, 
    "memory_manager": memory_manager,
} 


result = await run_session(runner, demo_query, session_id)


print("Pipeline executed successfully.") 



### Session: demo-session-001

User > Sleep disorder detection using multimodal physiological signals
Model: > Okay, Root Research Orchestrator here. The user wants to understand **"Sleep disorder detection using multimodal physiological signals."** This is a broad but critical area of research. I need to break this down into actionable research tasks for my specialized agents to ensure comprehensive coverage.

I will assign tasks focusing on current approaches, methodological details, specific applications, and data/ethical considerations.

**Here are the initial research tasks:**

1.  **Literature Reviewer (LR) Task:**
    *   **Objective:** Provide a comprehensive overview of the current landscape and most commonly utilized multimodal physiological signals for sleep disorder detection. Identify key trends, recent breakthroughs, and influential studies in the last 5-7 years.
    *   **Specific Questions:**
        *   What are the primary physiological signals (e.g., EEG, EOG, EMG, 

### 9.3 View Final Report

This prints the orchestrator’s structured output.

In [32]:
result

In [33]:
# @title View Final Report

from pprint import pprint

print("\n================ FINAL REPORT ================\n")
pprint(result["final_report"])



================ FINAL REPORT ================



TypeError: 'NoneType' object is not subscriptable

### 9.4 View Session Memory After Execution

This demonstrates that memory was updated—just like in the Day-3 memory notebooks.

In [34]:
# @title Memory Snapshot

session_state = session_service.get(session_id)
memory_after = session_state.get("memory", [])

print("\n================ SESSION MEMORY ================\n")
for i, item in enumerate(memory_after, 1):
    print(f"Memory Item {i}:", item)


AttributeError: 'InMemorySessionService' object has no attribute 'get'

# 📘 SECTION 10 — JSON Mode Output & Formatting
Clean extraction, validation, and human-readable formatting of the final agent output

## 🧾 Section 10: JSON Mode Output & Formatting

The final orchestrator output is an LLM-generated JSON-like structure.
In this section we:

1. Extract JSON from the model output
2. Validate and clean it
3. Provide a "pretty print" text version
4. Demonstrate the difference between raw machine output vs human-friendly rendering

This follows the output handling patterns shown in:
- Day-4A (Observability & Logging)
- Day-4B (Evaluation & Scoring)


### 10.1 Utility: Safe JSON Extraction

LLM outputs sometimes contain leading/trailing commentary.
This utility mirrors the JSON cleanup/validation helpers from Day-4 notebooks.

In [ ]:
# @title JSON Extraction Utility 

def extract_json_block(text: str):
    """
    Extracts the first JSON object found inside an LLM output,
    with liberal cleanup rules similar to ADK Day-4 evaluation utilities.
    """
    # Try strict load first
    try:
        return json.loads(text)
    except:
        pass

    # Regex fallback — extract braces
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        candidate = match.group(0)
        try:
            return json.loads(candidate)
        except:
            pass

    # Final fallback
    return {"error": "Could not parse JSON output", "raw_output": text}


### 10.2 Extract & Display Structured JSON

We take the output from Section 9 (result["final_report"])
and convert it into a validated Python dict.

In [ ]:
# @title Extract JSON from Final Report

raw_json_text = result.get("final_report", "")

clean_json = extract_json_block(raw_json_text)

print("======== CLEAN JSON OUTPUT ========\n")
from pprint import pprint
pprint(clean_json)


### 10.3 Pretty-Printed Human-Readable Summary

This creates a clean text summary from the JSON fields.

In [ ]:
# @title Pretty Printed Summary

def pretty_print_research_summary(j):
    if "error" in j:
        print("⚠️ JSON error:", j["error"])
        print("Raw output:\n", j.get("raw_output", ""))
        return
    
    topic = j.get("topic", "N/A")
    background = j.get("conceptual_background", "N/A")
    strengths = j.get("strong_points", [])
    weaknesses = j.get("weak_points", [])
    summary = j.get("overall_summary", "N/A")

    print(f"# 🧠 Research Topic\n{topic}\n")
    print("## 📘 Background\n", background, "\n")

    print("## 👍 Strengths")
    for s in strengths:
        print(" -", s)

    print("\n## ⚠️ Weaknesses / Gaps")
    for w in weaknesses:
        print(" -", w)

    print("\n## 📝 Final Summary\n", summary)


### 10.4 Generate The Human-Readable Version

In [ ]:
# @title Generate Research Summary

print("\n======== HUMAN-READABLE RESEARCH SUMMARY ========\n")
pretty_print_research_summary(clean_json)


# 📘 SECTION 11 — Appendix & Troubleshooting
Additional notes, debugging guidance, and extension tips

## 📚 Section 11: Appendix & Troubleshooting

This section provides references and troubleshooting instructions to help you
debug or extend the Smart Research Assistant. It mirrors the format used in the
appendix sections of the ADK Day-4/Day-5 notebooks.


### 🧩 Issue: "google_search" returns empty results
Possible causes:
- Query too specific
- DuckDuckGo/Google wrapper returns short or irrelevant entries
- Upstream search engine throttling

Fixes:
- Add more context words into the search query
- Re-run the orchestrator to update cached results
- Use broader phrasing ("overview of …", "recent research on …")


### 🧩 Issue: JSON extraction fails in Section 10
Likely cause:
- LLM returned natural language around the JSON
- JSON contains trailing commas or stylistic variations

Fixes:
- Use the JSON extraction helper to sanitize output
- Add a stricter instruction in ResearchSynthesizerAgent
- Reduce temperature for more deterministic output


### 🧩 Issue: Memory grows too large
MemoryManager compaction may not be triggered if:
- Too few memory items exist
- Items contain very long text blocks

Fixes:
- Reduce MAX_CONTEXT_TURNS or increase compaction frequency
- Add custom logic to trim long content inside add_memory_item()


### 11.2 Debugging Agent Outputs

### 🔍 Tip: Print intermediate agent outputs
Inside the orchestrator, you can temporarily print:

print("Q-agent:", q_result)
print("Web-findings:", web_output)
print("Concept explain:", explain_output)

This mirrors patterns used in Day-4 (Observability Tools) to debug pipelines.


### 🔍 Tip: Use verbose LLM logs
You can enable verbose logging of prompts/responses with:

Gemini(..., enable_prompt_logging=True)

This mirrors Day-4A where they show log-based observability.


### 11.3 Troubleshooting Parallel Execution

From Day-5A (Agent-to-Agent Communication):

### ⚡ If asyncio.gather hangs:
- Check that both web_agent.run() and explain_agent.run() are async
- Check if the google_search tool responded with a dict
- Ensure no blocking code inside agents


### ⚡ If one parallel agent fails:
asyncio.gather(..., return_exceptions=True)

Using this pattern allows partial recovery without crashing the orchestrator.


### 11.4 Extending the System

### ➕ Add a Literature Review Agent
Use:
- google_search
- summarization FunctionTool
- a new LLM agent to synthesize multi-source findings


### ➕ Add a Citation Retrieval Agent
- Wrap a scholarly search API into a FunctionTool
- Query terms extracted by the QueryUnderstandingAgent


### ➕ Add Model-Based Observability
Implement counters, timers, and JSON logs like Day-4A to inspect:
- time per agent
- token usage
- intermediate trace output


### 11.5 Deployment Guidance (Day-5B Style)

### 🚢 Deployment Notes
For production use:
1. Replace InMemoryRunner with a persistent session backend
2. Use an MCP-hosted search tool for robust research queries
3. Add retries/timeouts around all tool calls
4. Consider splitting agents across worker processes for scalability


### 🌐 API Deployment
Use:
- Vertex AI Agent Runtime
- A REST endpoint wrapping runner.run(...)
- JSON input/output following Section 10

This mirrors the deployment examples in Day-5B.
